In [59]:
import pandas as pd
import numpy as np
import polars as pl
import lib.models.baseline as baseline
import lib.eval as eval

import importlib
importlib.reload(eval)
importlib.reload(baseline)

<module 'lib.models.baseline' from '/Users/mathiasraa/Desktop/ntnu/recommender-systems/lib/models/baseline.py'>

In [61]:
behavior_polars_train = pl.read_csv("data/MINDlarge_train/behaviors.tsv", separator='\t', has_header=False)
behavior_polars_dev = pl.read_csv("data/MINDlarge_dev/behaviors.tsv", separator='\t', has_header=False)
behavior_polars_test = pl.read_csv("data/MINDlarge_test/behaviors.tsv", separator='\t', has_header=False)
behavior_polars_test.columns = ['impression_id', 'user_id', 'timestamp', 'history', 'impressions']
behavior_polars_train.columns = ['impression_id', 'user_id', 'timestamp', 'history', 'impressions']
behavior_polars_dev.columns = ['impression_id', 'user_id', 'timestamp', 'history', 'impressions']

In [62]:
baseline_predictions = baseline.create_popularity_baseline(behavior_polars_train, behavior_polars_dev)

In [63]:
normalized_predictions = baseline.normalize_scores(baseline_predictions)

In [64]:
normalized_predictions

impression_id,news_id,score
i64,str,f64
1,"""N91737""",9.4516e-8
1,"""N30206""",0.000961
1,"""N54368""",0.238337
1,"""N117802""",0.002433
1,"""N18190""",0.150057
…,…,…
376471,"""N86509""",0.0
376471,"""N12384""",0.003406
376471,"""N14719""",0.019876


In [65]:
results = eval.evaluate_mind_predictions(normalized_predictions, behavior_polars_dev)

In [66]:
pd.DataFrame(results, index=[0])

,auc,mrr,ndcg@5,ndcg@10
0,0.552853,0.261856,0.269713,0.332056
